# Wildfire Prediction

This Jupyter Notebook implements various MLPs and CNNs to predict whether a landscape image represents a wildfire or not. PyTorch will be the main library used for this project. 

Image Specifications:
- Type: `.jpg`
- Dimensions: 250 x 250 pixels
- Horizontal/vertical resolution: 96 dpi
- Bit depth: 24

In [1]:
# Import libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader

In [2]:
# Define transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [3]:
# Load the data
train_set = torchvision.datasets.ImageFolder('forest_fire/Training and Validation', transform)
trainloader = DataLoader(
    train_set,
    batch_size = 64,
    shuffle = True
)
test_set = torchvision.datasets.ImageFolder('forest_fire/Testing', transform)
testloader = DataLoader(
    test_set,
    batch_size = 64,
    shuffle = False
)

In [4]:
# Global constants
input_size = 250 * 250 * 3
classes = ['fire', 'nofire']
num_classes = 2
num_epochs = 20

# Eight different MLP architectures to experiment
mlp_hidden_sizes = [
    [64],
    [128, 64],
    [256, 64],
    [512, 128],
    [256, 128, 64],
    [1024, 256, 64],
    [512, 256, 128, 64],
    [1024, 512, 256, 128, 64]
]

# Nine different CNN architectures to experiment
conv_sizes = [3, 5, 7]
cnn_hidden_sizes = [
    [32],
    [32, 64],
    [32, 64, 128]
]

In [5]:
# Create a dynamic Multi-layer Perceptron
class DynamicMLP(nn.Module):
    
    def __init__(self, input_size, hidden_sizes, num_classes):
        super(DynamicMLP, self).__init__()
        layers = [nn.Flatten()]

        # Construct MLP
        prev_size = input_size
        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(prev_size, hidden_size))
            layers.append(nn.ReLU())
            prev_size = hidden_size

        # Make output layer
        layers.append(nn.Linear(prev_size, num_classes))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

In [6]:
# Create a dynamic Convolutional Neural Network
class DynamicCNN(nn.Module):
    
    def __init__(self, hidden_sizes, conv_size, num_classes):
        super(DynamicCNN, self).__init__()
        layers = []

        # Construct CNN
        curr_layer = 3 # initially RGB channels
        img_dim = 250 # image dimension
        for hidden_size in hidden_sizes:
            padding = conv_size // 2
            layers.append(nn.Conv2d(curr_layer, hidden_size, kernel_size=conv_size, padding=padding))
            layers.append(nn.ReLU())
            layers.append(nn.MaxPool2d(2))
            img_dim = img_dim // 2
            curr_layer = hidden_size

        # Output layer
        # print(img_dim * img_dim * curr_layer)
        layers.append(nn.Flatten())
        layers.append(nn.Linear(img_dim * img_dim * curr_layer, num_classes))

        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

In [12]:
# Training loop
def train_loop(train_model, trainloader, num_epochs):

    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model = train_model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        avg_loss = running_loss / len(trainloader)
        accuracy = 100 * correct / total

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")

In [13]:
# Create the MLPs
mlp_1 = DynamicMLP(input_size, mlp_hidden_sizes[0], num_classes)
mlp_2 = DynamicMLP(input_size, mlp_hidden_sizes[1], num_classes)
mlp_3 = DynamicMLP(input_size, mlp_hidden_sizes[2], num_classes)
mlp_4 = DynamicMLP(input_size, mlp_hidden_sizes[3], num_classes)
mlp_5 = DynamicMLP(input_size, mlp_hidden_sizes[4], num_classes)
mlp_6 = DynamicMLP(input_size, mlp_hidden_sizes[5], num_classes)
mlp_7 = DynamicMLP(input_size, mlp_hidden_sizes[6], num_classes)
mlp_8 = DynamicMLP(input_size, mlp_hidden_sizes[7], num_classes)

In [21]:
train_loop(mlp_1, trainloader, num_epochs)

Epoch [1/20], Loss: 3.5251, Accuracy: 84.01%
Epoch [2/20], Loss: 1.3195, Accuracy: 91.65%
Epoch [3/20], Loss: 0.4122, Accuracy: 95.14%
Epoch [4/20], Loss: 0.2582, Accuracy: 95.91%
Epoch [5/20], Loss: 0.1289, Accuracy: 97.60%
Epoch [6/20], Loss: 0.1001, Accuracy: 97.54%
Epoch [7/20], Loss: 0.0882, Accuracy: 97.49%
Epoch [8/20], Loss: 0.0348, Accuracy: 98.96%
Epoch [9/20], Loss: 0.0834, Accuracy: 98.36%
Epoch [10/20], Loss: 0.0388, Accuracy: 98.74%
Epoch [11/20], Loss: 0.0765, Accuracy: 97.93%
Epoch [12/20], Loss: 0.0242, Accuracy: 99.02%
Epoch [13/20], Loss: 0.0797, Accuracy: 97.05%
Epoch [14/20], Loss: 0.0725, Accuracy: 97.76%
Epoch [15/20], Loss: 0.0293, Accuracy: 99.18%
Epoch [16/20], Loss: 0.0143, Accuracy: 99.56%
Epoch [17/20], Loss: 0.0180, Accuracy: 99.51%
Epoch [18/20], Loss: 0.0154, Accuracy: 99.56%
Epoch [19/20], Loss: 0.0096, Accuracy: 99.73%
Epoch [20/20], Loss: 0.0184, Accuracy: 99.51%


In [22]:
train_loop(mlp_2, trainloader, num_epochs)

Epoch [1/20], Loss: 1.1893, Accuracy: 83.52%
Epoch [2/20], Loss: 0.3106, Accuracy: 92.14%
Epoch [3/20], Loss: 0.1643, Accuracy: 94.76%
Epoch [4/20], Loss: 0.1222, Accuracy: 96.40%
Epoch [5/20], Loss: 0.0790, Accuracy: 97.76%
Epoch [6/20], Loss: 0.0816, Accuracy: 97.49%
Epoch [7/20], Loss: 0.0666, Accuracy: 97.54%
Epoch [8/20], Loss: 0.0546, Accuracy: 98.20%
Epoch [9/20], Loss: 0.0294, Accuracy: 99.02%
Epoch [10/20], Loss: 0.0190, Accuracy: 99.73%
Epoch [11/20], Loss: 0.0209, Accuracy: 99.45%
Epoch [12/20], Loss: 0.0140, Accuracy: 99.56%
Epoch [13/20], Loss: 0.0083, Accuracy: 99.89%
Epoch [14/20], Loss: 0.0074, Accuracy: 99.78%
Epoch [15/20], Loss: 0.0288, Accuracy: 98.91%
Epoch [16/20], Loss: 0.0163, Accuracy: 99.51%
Epoch [17/20], Loss: 0.0385, Accuracy: 98.96%
Epoch [18/20], Loss: 0.1352, Accuracy: 96.89%
Epoch [19/20], Loss: 0.1827, Accuracy: 96.12%
Epoch [20/20], Loss: 0.1401, Accuracy: 96.78%


In [23]:
train_loop(mlp_3, trainloader, num_epochs)

Epoch [1/20], Loss: 1.5434, Accuracy: 82.91%
Epoch [2/20], Loss: 0.4562, Accuracy: 91.98%
Epoch [3/20], Loss: 0.1631, Accuracy: 94.87%
Epoch [4/20], Loss: 0.1344, Accuracy: 96.23%
Epoch [5/20], Loss: 0.1329, Accuracy: 97.11%
Epoch [6/20], Loss: 0.0706, Accuracy: 97.82%
Epoch [7/20], Loss: 0.0374, Accuracy: 99.02%
Epoch [8/20], Loss: 0.0551, Accuracy: 98.25%
Epoch [9/20], Loss: 0.0919, Accuracy: 97.16%
Epoch [10/20], Loss: 0.0785, Accuracy: 97.65%
Epoch [11/20], Loss: 0.1148, Accuracy: 96.78%
Epoch [12/20], Loss: 0.0443, Accuracy: 98.69%
Epoch [13/20], Loss: 0.0253, Accuracy: 99.24%
Epoch [14/20], Loss: 0.0299, Accuracy: 99.29%
Epoch [15/20], Loss: 0.0132, Accuracy: 99.78%
Epoch [16/20], Loss: 0.0064, Accuracy: 99.95%
Epoch [17/20], Loss: 0.0106, Accuracy: 99.67%
Epoch [18/20], Loss: 0.0078, Accuracy: 99.84%
Epoch [19/20], Loss: 0.0074, Accuracy: 99.95%
Epoch [20/20], Loss: 0.0024, Accuracy: 99.95%


In [24]:
train_loop(mlp_4, trainloader, num_epochs)

Epoch [1/20], Loss: 3.3105, Accuracy: 81.99%
Epoch [2/20], Loss: 0.5806, Accuracy: 92.25%
Epoch [3/20], Loss: 0.2708, Accuracy: 93.50%
Epoch [4/20], Loss: 0.2578, Accuracy: 93.78%
Epoch [5/20], Loss: 0.0964, Accuracy: 97.16%
Epoch [6/20], Loss: 0.0716, Accuracy: 97.98%
Epoch [7/20], Loss: 0.0746, Accuracy: 97.76%
Epoch [8/20], Loss: 0.0881, Accuracy: 97.22%
Epoch [9/20], Loss: 0.0514, Accuracy: 98.25%
Epoch [10/20], Loss: 0.0292, Accuracy: 99.24%
Epoch [11/20], Loss: 0.0159, Accuracy: 99.34%
Epoch [12/20], Loss: 0.0168, Accuracy: 99.51%
Epoch [13/20], Loss: 0.0167, Accuracy: 99.56%
Epoch [14/20], Loss: 0.0045, Accuracy: 99.89%
Epoch [15/20], Loss: 0.0036, Accuracy: 99.95%
Epoch [16/20], Loss: 0.0021, Accuracy: 100.00%
Epoch [17/20], Loss: 0.0011, Accuracy: 100.00%
Epoch [18/20], Loss: 0.0010, Accuracy: 100.00%
Epoch [19/20], Loss: 0.0008, Accuracy: 100.00%
Epoch [20/20], Loss: 0.0007, Accuracy: 100.00%


In [25]:
train_loop(mlp_5, trainloader, num_epochs)

Epoch [1/20], Loss: 0.6663, Accuracy: 83.68%
Epoch [2/20], Loss: 0.2004, Accuracy: 92.85%
Epoch [3/20], Loss: 0.1609, Accuracy: 94.71%
Epoch [4/20], Loss: 0.1248, Accuracy: 96.07%
Epoch [5/20], Loss: 0.1409, Accuracy: 95.41%
Epoch [6/20], Loss: 0.1678, Accuracy: 95.69%
Epoch [7/20], Loss: 0.1039, Accuracy: 97.33%
Epoch [8/20], Loss: 0.0512, Accuracy: 98.74%
Epoch [9/20], Loss: 0.0323, Accuracy: 99.18%
Epoch [10/20], Loss: 0.0286, Accuracy: 99.02%
Epoch [11/20], Loss: 0.0210, Accuracy: 99.51%
Epoch [12/20], Loss: 0.0154, Accuracy: 99.45%
Epoch [13/20], Loss: 0.0147, Accuracy: 99.73%
Epoch [14/20], Loss: 0.0228, Accuracy: 99.40%
Epoch [15/20], Loss: 0.0585, Accuracy: 98.31%
Epoch [16/20], Loss: 0.0730, Accuracy: 97.43%
Epoch [17/20], Loss: 0.0890, Accuracy: 97.71%
Epoch [18/20], Loss: 0.0761, Accuracy: 97.43%
Epoch [19/20], Loss: 0.0345, Accuracy: 98.64%
Epoch [20/20], Loss: 0.0197, Accuracy: 99.24%


In [26]:
train_loop(mlp_6, trainloader, num_epochs)

Epoch [1/20], Loss: 1.5800, Accuracy: 79.31%
Epoch [2/20], Loss: 0.2579, Accuracy: 91.38%
Epoch [3/20], Loss: 0.2030, Accuracy: 93.29%
Epoch [4/20], Loss: 0.1689, Accuracy: 94.81%
Epoch [5/20], Loss: 0.1867, Accuracy: 93.29%
Epoch [6/20], Loss: 0.0867, Accuracy: 97.65%
Epoch [7/20], Loss: 0.0533, Accuracy: 98.96%
Epoch [8/20], Loss: 0.0583, Accuracy: 98.58%
Epoch [9/20], Loss: 0.0643, Accuracy: 97.98%
Epoch [10/20], Loss: 0.0590, Accuracy: 98.31%
Epoch [11/20], Loss: 0.0414, Accuracy: 98.58%
Epoch [12/20], Loss: 0.0202, Accuracy: 99.62%
Epoch [13/20], Loss: 0.0087, Accuracy: 99.89%
Epoch [14/20], Loss: 0.0044, Accuracy: 99.95%
Epoch [15/20], Loss: 0.0031, Accuracy: 99.95%
Epoch [16/20], Loss: 0.0032, Accuracy: 99.95%
Epoch [17/20], Loss: 0.0025, Accuracy: 100.00%
Epoch [18/20], Loss: 0.0147, Accuracy: 99.51%
Epoch [19/20], Loss: 0.0988, Accuracy: 97.76%
Epoch [20/20], Loss: 0.0453, Accuracy: 98.74%


In [27]:
train_loop(mlp_7, trainloader, num_epochs)

Epoch [1/20], Loss: 0.7929, Accuracy: 79.97%
Epoch [2/20], Loss: 0.2292, Accuracy: 91.48%
Epoch [3/20], Loss: 0.1662, Accuracy: 94.05%
Epoch [4/20], Loss: 0.1729, Accuracy: 94.38%
Epoch [5/20], Loss: 0.1131, Accuracy: 96.07%
Epoch [6/20], Loss: 0.0842, Accuracy: 97.22%
Epoch [7/20], Loss: 0.0627, Accuracy: 97.82%
Epoch [8/20], Loss: 0.0750, Accuracy: 97.93%
Epoch [9/20], Loss: 0.0446, Accuracy: 98.74%
Epoch [10/20], Loss: 0.0407, Accuracy: 98.85%
Epoch [11/20], Loss: 0.0617, Accuracy: 98.14%
Epoch [12/20], Loss: 0.0501, Accuracy: 98.42%
Epoch [13/20], Loss: 0.0356, Accuracy: 98.91%
Epoch [14/20], Loss: 0.0180, Accuracy: 99.56%
Epoch [15/20], Loss: 0.0042, Accuracy: 99.89%
Epoch [16/20], Loss: 0.1624, Accuracy: 94.71%
Epoch [17/20], Loss: 0.1361, Accuracy: 97.05%
Epoch [18/20], Loss: 0.0860, Accuracy: 98.14%
Epoch [19/20], Loss: 0.0442, Accuracy: 98.36%
Epoch [20/20], Loss: 0.0260, Accuracy: 99.18%


In [28]:
train_loop(mlp_8, trainloader, num_epochs)

Epoch [1/20], Loss: 0.5094, Accuracy: 80.35%
Epoch [2/20], Loss: 0.2363, Accuracy: 90.61%
Epoch [3/20], Loss: 0.1679, Accuracy: 93.67%
Epoch [4/20], Loss: 0.1350, Accuracy: 95.20%
Epoch [5/20], Loss: 0.1482, Accuracy: 94.98%
Epoch [6/20], Loss: 0.0894, Accuracy: 97.00%
Epoch [7/20], Loss: 0.1540, Accuracy: 95.31%
Epoch [8/20], Loss: 0.0998, Accuracy: 96.56%
Epoch [9/20], Loss: 0.0766, Accuracy: 97.60%
Epoch [10/20], Loss: 0.1085, Accuracy: 95.58%
Epoch [11/20], Loss: 0.0715, Accuracy: 97.71%
Epoch [12/20], Loss: 0.0510, Accuracy: 98.31%
Epoch [13/20], Loss: 0.0413, Accuracy: 98.42%
Epoch [14/20], Loss: 0.0925, Accuracy: 98.53%
Epoch [15/20], Loss: 0.2039, Accuracy: 94.05%
Epoch [16/20], Loss: 0.1402, Accuracy: 96.40%
Epoch [17/20], Loss: 0.1088, Accuracy: 98.25%
Epoch [18/20], Loss: 0.1035, Accuracy: 98.42%
Epoch [19/20], Loss: 0.0559, Accuracy: 98.31%
Epoch [20/20], Loss: 0.0505, Accuracy: 99.13%


In [29]:
# Create the CNNs

# Architecture 1
cnn_1 = DynamicCNN(cnn_hidden_sizes[0], conv_sizes[0], num_classes)
cnn_2 = DynamicCNN(cnn_hidden_sizes[0], conv_sizes[1], num_classes)
cnn_3 = DynamicCNN(cnn_hidden_sizes[0], conv_sizes[2], num_classes)

# Architecture 2
cnn_4 = DynamicCNN(cnn_hidden_sizes[1], conv_sizes[0], num_classes)
cnn_5 = DynamicCNN(cnn_hidden_sizes[1], conv_sizes[1], num_classes)
cnn_6 = DynamicCNN(cnn_hidden_sizes[1], conv_sizes[2], num_classes)

# Architecture 3
cnn_7 = DynamicCNN(cnn_hidden_sizes[2], conv_sizes[0], num_classes)
cnn_8 = DynamicCNN(cnn_hidden_sizes[2], conv_sizes[1], num_classes)
cnn_9 = DynamicCNN(cnn_hidden_sizes[2], conv_sizes[2], num_classes)

In [ ]:
train_loop(cnn_1, trainloader, num_epochs)

In [ ]:
train_loop(cnn_2, trainloader, num_epochs)

In [ ]:
train_loop(cnn_3, trainloader, num_epochs)

In [ ]:
train_loop(cnn_4, trainloader, num_epochs)

In [ ]:
train_loop(cnn_5, trainloader, num_epochs)

In [ ]:
train_loop(cnn_6, trainloader, num_epochs)

In [ ]:
train_loop(cnn_7, trainloader, num_epochs)

In [ ]:
train_loop(cnn_8, trainloader, num_epochs)

In [ ]:
train_loop(cnn_9, trainloader, num_epochs)